In [1]:
import time
import warnings
import numpy as np

warnings.simplefilter("ignore")

import torch
import torchvision
from torch import nn, optim
import torch.multiprocessing as mp
import torchvision.transforms as transforms

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager

from tqdm.notebook import tqdm 


if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [2]:
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [3]:
# Hyperparameters for training and privacy
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5

EPOCHS = 20
LR = 1e-3

In [4]:
# Function to make the model differentially private
def make_private(model, optimizer, train_loader):
    model_dp, optimizer_dp, train_loader_dp = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=EPOCHS,
        target_epsilon=EPSILON,
        target_delta=DELTA,
        max_grad_norm=MAX_GRAD_NORM,
    )
    return model_dp, optimizer_dp, train_loader_dp

# Function to calculate accuracy
def accuracy(preds, labels):
    return (preds == labels).mean()

# Function for training the model
def train(model, optimizer, train_loader, device, dp, task): 
    start_time = time.time()

    model = model.to(device)

    for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    
        model.train()
        criterion = nn.CrossEntropyLoss()

        losses = []
        accs = []

        # BatchMemoryManager manages the memory usage (e.g. you can use bigger logical batches)
        with BatchMemoryManager(data_loader=train_loader, max_physical_batch_size=128, optimizer=optimizer) as loader:
            if not dp:
                loader = train_loader

            for i, (images, target) in enumerate(loader):   
                optimizer.zero_grad()
                images = images.to(device)
                target = target.to(device)
                
                if task == 'multi_class': 
                    target = target.squeeze().float()
                
                output = model(images)
                loss = criterion(output, target)

                preds = np.argmax(output.detach().cpu().numpy(), axis=1)
                labels = target.detach().cpu().numpy()

                acc = accuracy(preds, labels)

                losses.append(loss.item())
                accs.append(acc)

                loss.backward()
                optimizer.step()

            if dp:
                epsilon = privacy_engine.get_epsilon(DELTA)
            else:
                epsilon = float('inf')
            print(
                f"Training Epoch {epoch+1:02d} \t"
                f"Loss: {np.mean(losses):.6f} | "
                f"Acc: {np.mean(accs) * 100:.6f} | "
                f"ε = {epsilon:.2f}"
            )
                
    end_time = time.time()
    training_time = end_time - start_time
    print(f'\nTraining Time \t\t{training_time:.2f} seconds')
                
# Function for testing the model
def test(model, test_loader, device, dp, task):
    model = model.to(device)
    
    model.eval()
    criterion = nn.CrossEntropyLoss()
        
    losses = []
    accs = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            if task == 'multi_class': 
                target = target.squeeze().float()
                
            output = model(images)
            loss = criterion(output, target)
            
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            accs.append(acc)

    print(
        f"Test Set \t\t"
        f"Loss: {np.mean(losses):.6f} | "
        f"Acc: {np.mean(accs) * 100:.6f} "
    )

# Function for the overall pipeline
def pipeline(model, train_loader, test_loader, device, task=None):
    optimizer = optim.RMSprop(model.parameters(), lr=LR)

    # Specific layers of the model can't be made differentially private, so the model has to be fixed
    model_fixed = ModuleValidator.fix(model)
    optimizer_fixed = optim.RMSprop(model_fixed.parameters(), lr=LR)

    model_dp, optimizer_dp, train_loader_dp = make_private(model_fixed, optimizer_fixed, train_loader)
    
    print(f'\n===== With DP =====')
    train(model_dp, optimizer_dp, train_loader_dp, device, True, task)
    test(model_dp, test_loader, device, True, task)
    print(f'\n===== Without DP =====')
    train(model, optimizer, train_loader, device, False, task)
    test(model, test_loader, device, False, task)

In [6]:
# CIFAR 10
train_dataset = torchvision.datasets.CIFAR10(root='/Users/valentinbiller/Downloads/data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='/Users/valentinbiller/Downloads/data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

# Initialize the privacy engine and the model
privacy_engine = PrivacyEngine()
model = torchvision.models.resnet18(num_classes=10)

pipeline(model, train_loader, test_loader, device)

Files already downloaded and verified
Files already downloaded and verified

===== With DP =====


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 2.044835 | Acc: 29.813140 | ε = 15.40
Training Epoch 02 	Loss: 1.731952 | Acc: 44.228565 | ε = 18.97
Training Epoch 03 	Loss: 1.744378 | Acc: 47.843522 | ε = 21.78
Training Epoch 04 	Loss: 1.725699 | Acc: 49.884614 | ε = 24.21
Training Epoch 05 	Loss: 1.716553 | Acc: 51.629311 | ε = 26.40
Training Epoch 06 	Loss: 1.690524 | Acc: 53.580725 | ε = 28.43
Training Epoch 07 	Loss: 1.707080 | Acc: 54.283086 | ε = 30.34
Training Epoch 08 	Loss: 1.673652 | Acc: 55.519441 | ε = 32.14
Training Epoch 09 	Loss: 1.690878 | Acc: 55.878178 | ε = 33.87
Training Epoch 10 	Loss: 1.661158 | Acc: 56.794265 | ε = 35.54
Training Epoch 11 	Loss: 1.664296 | Acc: 57.527864 | ε = 37.14
Training Epoch 12 	Loss: 1.645446 | Acc: 58.255640 | ε = 38.70
Training Epoch 13 	Loss: 1.658301 | Acc: 58.339165 | ε = 40.22
Training Epoch 14 	Loss: 1.642412 | Acc: 59.203316 | ε = 41.70
Training Epoch 15 	Loss: 1.640699 | Acc: 59.127673 | ε = 43.15
Training Epoch 16 	Loss: 1.618777 | Acc: 60.014105 | ε 

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.555629 | Acc: 44.900351 | ε = inf
Training Epoch 02 	Loss: 1.050227 | Acc: 62.592474 | ε = inf
Training Epoch 03 	Loss: 0.841995 | Acc: 70.547672 | ε = inf
Training Epoch 04 	Loss: 0.693986 | Acc: 75.778460 | ε = inf
Training Epoch 05 	Loss: 0.571016 | Acc: 80.101642 | ε = inf
Training Epoch 06 	Loss: 0.463993 | Acc: 83.962054 | ε = inf
Training Epoch 07 	Loss: 0.368484 | Acc: 87.204241 | ε = inf
Training Epoch 08 	Loss: 0.285049 | Acc: 90.141901 | ε = inf
Training Epoch 09 	Loss: 0.226170 | Acc: 92.120536 | ε = inf
Training Epoch 10 	Loss: 0.184158 | Acc: 93.557079 | ε = inf
Training Epoch 11 	Loss: 0.150194 | Acc: 94.693080 | ε = inf
Training Epoch 12 	Loss: 0.132126 | Acc: 95.380261 | ε = inf
Training Epoch 13 	Loss: 0.116747 | Acc: 96.020807 | ε = inf
Training Epoch 14 	Loss: 0.109167 | Acc: 96.247608 | ε = inf
Training Epoch 15 	Loss: 0.089109 | Acc: 96.926419 | ε = inf
Training Epoch 16 	Loss: 0.085142 | Acc: 96.988202 | ε = inf
Training Epoch 17 	Loss:

In [5]:
import medmnist
from medmnist import INFO

root = '/Users/valentinbiller/Downloads/'
datasets = ['dermamnist', 'pneumoniamnist', 'bloodmnist', 'organcmnist']

transformMedMNIST = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]),
])

for data_flag in datasets:
    
    info = INFO[data_flag]
    DataClass = getattr(medmnist, info['python_class'])
    
    train_dataset = DataClass(split='train', transform=transformMedMNIST, download=True, root=root)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

    test_dataset = DataClass(split='test', transform=transformMedMNIST, download=True, root=root)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
    
    num_classes = len(info['label'])
    
    privacy_engine = PrivacyEngine()
    model = torchvision.models.resnet18(num_classes=num_classes)
    if data_flag == 'pneumoniamnist' or data_flag == 'organcmnist':     
        model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 
    print(f'\n\n\n ========== {data_flag} ========== \n')
    pipeline(model, train_loader, test_loader, device, task='multi_class')
    print(f'\n\n\n')

Using downloaded and verified file: /Users/valentinbiller/Downloads/dermamnist.npz
Using downloaded and verified file: /Users/valentinbiller/Downloads/dermamnist.npz



 ========== dermamnist ========== 


===== With DP =====


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 2.551348 | Acc: 60.523589 | ε = 14.26
Training Epoch 02 	Loss: 1.902619 | Acc: 67.368585 | ε = 17.88
Training Epoch 03 	Loss: 1.821546 | Acc: 66.446917 | ε = 20.74
Training Epoch 04 	Loss: 1.524055 | Acc: 67.763645 | ε = 23.22
Training Epoch 05 	Loss: 1.476889 | Acc: 68.231209 | ε = 25.48
Training Epoch 06 	Loss: 1.380161 | Acc: 69.751790 | ε = 27.57
Training Epoch 07 	Loss: 1.362810 | Acc: 70.174934 | ε = 29.53
Training Epoch 08 	Loss: 1.312899 | Acc: 70.674299 | ε = 31.40
Training Epoch 09 	Loss: 1.271073 | Acc: 71.304581 | ε = 33.19
Training Epoch 10 	Loss: 1.287653 | Acc: 70.828164 | ε = 34.92
Training Epoch 11 	Loss: 1.186437 | Acc: 72.439810 | ε = 36.59
Training Epoch 12 	Loss: 1.288666 | Acc: 71.950280 | ε = 38.21
Training Epoch 13 	Loss: 1.199026 | Acc: 73.454015 | ε = 39.79
Training Epoch 14 	Loss: 1.248781 | Acc: 71.854859 | ε = 41.33
Training Epoch 15 	Loss: 1.176952 | Acc: 72.586511 | ε = 42.84
Training Epoch 16 	Loss: 1.161274 | Acc: 73.133663 | ε 

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.193251 | Acc: 64.842869 | ε = inf
Training Epoch 02 	Loss: 0.862709 | Acc: 69.609816 | ε = inf
Training Epoch 03 	Loss: 0.814285 | Acc: 70.563910 | ε = inf
Training Epoch 04 	Loss: 0.769928 | Acc: 71.698191 | ε = inf
Training Epoch 05 	Loss: 0.739141 | Acc: 72.830563 | ε = inf
Training Epoch 06 	Loss: 0.717317 | Acc: 73.730762 | ε = inf
Training Epoch 07 	Loss: 0.686837 | Acc: 74.512747 | ε = inf
Training Epoch 08 	Loss: 0.647904 | Acc: 75.310297 | ε = inf
Training Epoch 09 	Loss: 0.619038 | Acc: 76.884839 | ε = inf
Training Epoch 10 	Loss: 0.601004 | Acc: 77.436266 | ε = inf
Training Epoch 11 	Loss: 0.561596 | Acc: 78.799195 | ε = inf
Training Epoch 12 	Loss: 0.537862 | Acc: 79.315378 | ε = inf
Training Epoch 13 	Loss: 0.499165 | Acc: 81.662653 | ε = inf
Training Epoch 14 	Loss: 0.486944 | Acc: 81.894384 | ε = inf
Training Epoch 15 	Loss: 0.425844 | Acc: 84.006403 | ε = inf
Training Epoch 16 	Loss: 0.388437 | Acc: 85.708412 | ε = inf
Training Epoch 17 	Loss:

...

Training Epoch 01 	Loss: 1.214864 | Acc: 63.926524 | ε = inf
Training Epoch 02 	Loss: 0.481730 | Acc: 82.049780 | ε = inf
Training Epoch 03 	Loss: 0.393518 | Acc: 85.402542 | ε = inf
Training Epoch 04 	Loss: 0.474790 | Acc: 83.140178 | ε = inf
Training Epoch 05 	Loss: 0.292580 | Acc: 89.351801 | ε = inf
Training Epoch 06 	Loss: 0.265034 | Acc: 90.287739 | ε = inf
Training Epoch 07 	Loss: 0.243841 | Acc: 91.132163 | ε = inf
Training Epoch 08 	Loss: 0.209984 | Acc: 92.262391 | ε = inf
Training Epoch 09 	Loss: 0.206671 | Acc: 92.420349 | ε = inf
Training Epoch 10 	Loss: 0.176496 | Acc: 93.392756 | ε = inf
Training Epoch 11 	Loss: 0.166425 | Acc: 93.924625 | ε = inf
Training Epoch 12 	Loss: 0.139576 | Acc: 95.088189 | ε = inf
Training Epoch 13 	Loss: 0.181882 | Acc: 93.829887 | ε = inf
Training Epoch 14 	Loss: 0.192262 | Acc: 93.725157 | ε = inf
Training Epoch 15 	Loss: 0.097747 | Acc: 96.521026 | ε = inf
Training Epoch 16 	Loss: 0.092845 | Acc: 96.587561 | ε = inf
Training Epoch 17 	Loss:

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 2.228463 | Acc: 38.767714 | ε = 14.66
Training Epoch 02 	Loss: 0.869041 | Acc: 74.596796 | ε = 18.26
Training Epoch 03 	Loss: 0.708963 | Acc: 80.844635 | ε = 21.10
Training Epoch 04 	Loss: 0.673098 | Acc: 82.360853 | ε = 23.57
Training Epoch 05 	Loss: 0.616290 | Acc: 84.243866 | ε = 25.80
Training Epoch 06 	Loss: 0.595377 | Acc: 85.278052 | ε = 27.87
Training Epoch 07 	Loss: 0.536857 | Acc: 86.509465 | ε = 29.81
Training Epoch 08 	Loss: 0.535025 | Acc: 86.628274 | ε = 31.66
Training Epoch 09 	Loss: 0.542969 | Acc: 86.348792 | ε = 33.43
Training Epoch 10 	Loss: 0.520541 | Acc: 87.486331 | ε = 35.13
Training Epoch 11 	Loss: 0.507330 | Acc: 87.477531 | ε = 36.78
Training Epoch 12 	Loss: 0.479737 | Acc: 88.163988 | ε = 38.38
Training Epoch 13 	Loss: 0.454631 | Acc: 88.638618 | ε = 39.94
Training Epoch 14 	Loss: 0.473386 | Acc: 88.325172 | ε = 41.46
Training Epoch 15 	Loss: 0.467767 | Acc: 88.289717 | ε = 42.95
Training Epoch 16 	Loss: 0.430540 | Acc: 89.270243 | ε 

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training Epoch 01 	Loss: 1.030646 | Acc: 68.747855 | ε = inf
Training Epoch 02 	Loss: 0.285904 | Acc: 89.772059 | ε = inf
Training Epoch 03 	Loss: 0.203161 | Acc: 92.671569 | ε = inf
Training Epoch 04 	Loss: 0.146550 | Acc: 94.844669 | ε = inf
Training Epoch 05 	Loss: 0.125356 | Acc: 95.740502 | ε = inf
Training Epoch 06 	Loss: 0.093434 | Acc: 96.848958 | ε = inf
Training Epoch 07 	Loss: 0.096155 | Acc: 96.687500 | ε = inf
Training Epoch 08 	Loss: 0.053884 | Acc: 98.112745 | ε = inf
Training Epoch 09 	Loss: 0.078744 | Acc: 97.717831 | ε = inf
Training Epoch 10 	Loss: 0.056143 | Acc: 98.257353 | ε = inf
Training Epoch 11 	Loss: 0.042073 | Acc: 98.628983 | ε = inf
Training Epoch 12 	Loss: 0.062661 | Acc: 98.048100 | ε = inf
Training Epoch 13 	Loss: 0.100159 | Acc: 97.837929 | ε = inf
Training Epoch 14 	Loss: 0.034725 | Acc: 98.825980 | ε = inf
Training Epoch 15 	Loss: 0.021387 | Acc: 99.304228 | ε = inf
Training Epoch 16 	Loss: 0.045226 | Acc: 98.594056 | ε = inf
Training Epoch 17 	Loss: